# pytorch基本操作

## 1.0 随机种子

In [1]:
# coding=utf-8
import torch
from torch import nn
import numpy as np
import random

SEED = 1234


# 设置随机种子
def randomSeed(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True


randomSeed(SEED)
# hyper parameters
in_dim = 1
n_hidden_1 = 1
n_hidden_2 = 1
out_dim = 1

/Users/bruce/anaconda3/envs/pytorch/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2.0 module 和children 区别

In [2]:
# hyper parameters
in_dim = 1
n_hidden_1 = 1
n_hidden_2 = 1
out_dim = 1
"""
module 和 children区别
module 是深度优先遍历打印出网络结构,而 children是只打印出网络的子结构,不再管子结构的下一结构
"""
class Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super().__init__()

        self.layer = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True),
        )
        self.layer3 = nn.Linear(n_hidden_2, out_dim)

        print("{:=^50s}".format("children"))
        for i, module in enumerate(self.children()):
            print(i, module)

        print("{:=^50s}".format("modules"))        
        for i, module in enumerate(self.modules()):
            print(i, module)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x


model = Net(in_dim, n_hidden_1, n_hidden_2, out_dim)


=====================children=====================
0 Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
  (1): ReLU(inplace=True)
)
1 Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
  (1): ReLU(inplace=True)
)
2 Linear(in_features=1, out_features=1, bias=True)
=====================modules======================
0 Net(
  (layer): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): ReLU(inplace=True)
  )
  (layer3): Linear(in_features=1, out_features=1, bias=True)
)
1 Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
  (1): ReLU(inplace=True)
)
2 Linear(in_features=1, out_features=1, bias=True)
3 ReLU(inplace=True)
4 Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
  (1): ReLU(inplace=True)
)
5 Linear(in_features=1, out_features=1, bias=True)
6 ReLU(inplace=True)
7 Linear(in_fe

## 3.0 向量

In [15]:
"""
embedding vector
"""
word_to_ix = {'hello': 1, 'world': 2}
# padding_idx (python:int, optional) – 填充id，比如，输入长度为100，但是每次的句子长度并不一样，后面就需要用统一的数字填充，
# 而这里就是指定这个数字，这样，网络在遇到填充id时，就不会计算其与其它符号的相关性。（初始化为0）
embeds = nn.Embedding(7, 5, padding_idx=0)
hello_idx = torch.LongTensor([word_to_ix['hello']])
hello_idx

tensor([1])

In [16]:
hello_embed = embeds(hello_idx)
print(hello_embed)
print(hello_embed.shape)

# 1,3,5维度
print(embeds(torch.LongTensor([1, 4, 3])))

# 2,3,5
print(embeds(torch.LongTensor([[1, 4, 3], [2, 3, 1]])).shape)

tensor([[-0.2504,  0.2037, -0.6473,  1.2173,  0.6089]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 5])
tensor([[-0.2504,  0.2037, -0.6473,  1.2173,  0.6089],
        [-0.0959,  0.9456, -1.2708,  0.2959,  0.1503],
        [ 0.4238, -0.6420,  1.2667, -0.0799, -0.3857]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 3, 5])


In [19]:
inputs = torch.randint(1, 7, (3, 3))
print(embeds(inputs).shape)
# print(embeds(inputs))

weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding = nn.Embedding.from_pretrained(weight)
print('embedding is ',embedding)
input = torch.LongTensor([1])
print(embedding(input))

torch.Size([3, 3, 5])
embedding is  Embedding(2, 3)
tensor([[4.0000, 5.1000, 6.3000]])


## 4.0 常用函数

In [52]:
"""
torch.mm  矩阵乘法
torch.bmm 三维矩阵乘法，第一维是batch size，比如[3,4,5] * [3,5,6] = [3,4,6]
torch.matmul 广播乘法,1维的话返回标量,2维的话返回矩阵乘法结果,其他情况比较复杂,具有广播功能
torch.mul 对位相乘,两个矩阵的维度必须一致才可以
torch.dot
"""


a = torch.Tensor([[1, 2], [2, 3]])
b = torch.Tensor([[2, 3], [2, 3]])
c = torch.mul(a, b)
print('对位相乘, a 和 b 维度必须相同， mul is:\n', c)
d = torch.mm(a, b)
print("矩阵乘法 mm is:\n", d)
e = torch.matmul(a, b)

print("{:=^50s} \n{}".format("广播乘法 matmul", e))


a1 = torch.Tensor([1, 2, 3])
a2 = torch.Tensor([1, 2, 3])
f = torch.dot(a1, a2)
print('一维点乘 dot is:\n', f)

对位相乘, a 和 b 维度必须相同， mul is:
 tensor([[2., 6.],
        [4., 9.]])
矩阵乘法 mm is:
 tensor([[ 6.,  9.],
        [10., 15.]])
===================广播乘法 matmul==================== 
tensor([[ 6.,  9.],
        [10., 15.]])
一维点乘 dot is:
 tensor(14.)


In [21]:
"""
torch.eq函数
"""
a = torch.FloatTensor([1, 2, 3])
b = torch.FloatTensor([2, 21, 3])
print(b.eq(a.data).cpu())
# output is [false, false, True]

tensor([False, False,  True])


In [24]:
"""
print 格式化输出
"""
import math
print('{0:n}'.format(20.000))
train_loss = 0.8797
print(f'Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')  # 7 前面是空格
print('Train loss: {:.3f} | train ppl: {:7.3f}'.format(train_loss, math.exp(train_loss)))



20
Train Loss: 0.880 | Train PPL:   2.410
Train loss: 0.880 | train ppl:   2.410


In [25]:
"""
masked_fill， 使用mask 技巧
"""
# import torch.nn.functional as F
# import numpy as np

a = torch.Tensor([1, 2, 3, 4])
a = a.masked_fill(mask=torch.ByteTensor([1, 1, 0, 0]), value=-np.inf)
print(a)
b = F.softmax(a, dim=0)
print(b)

tensor([-inf, -inf, 3., 4.])
tensor([0.0000, 0.0000, 0.2689, 0.7311])


/Users/bruce/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1640811914285/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1273.)
  


In [26]:
print(-np.inf)
print("#"*10)
a = torch.tensor([[[5, 5, 5, 5], [6, 6, 6, 6], [7, 7, 7, 7]], [[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3]]])
print(a)
print(a.size())
mask = torch.ByteTensor([[[0]], [[1]]])
print(mask.size())
b = a.masked_fill(mask, value=torch.tensor(-1e9))
print(b)
print(b.size())
print("#"*10)

-inf
##########
tensor([[[5, 5, 5, 5],
         [6, 6, 6, 6],
         [7, 7, 7, 7]],

        [[1, 1, 1, 1],
         [2, 2, 2, 2],
         [3, 3, 3, 3]]])
torch.Size([2, 3, 4])
torch.Size([2, 1, 1])
tensor([[[          5,           5,           5,           5],
         [          6,           6,           6,           6],
         [          7,           7,           7,           7]],

        [[-1000000000, -1000000000, -1000000000, -1000000000],
         [-1000000000, -1000000000, -1000000000, -1000000000],
         [-1000000000, -1000000000, -1000000000, -1000000000]]])
torch.Size([2, 3, 4])
##########


/Users/bruce/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1640811914285/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1273.)
  


In [3]:
a = torch.tensor([[[5, 5, 5, 5], [6, 6, 6, 6], [7, 7, 7, 7]], [[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3]]])
print(a)
print(a.size())
mask = torch.ByteTensor([[[1], [1], [0]], [[0], [1], [1]]])
print(mask.size())
b = a.masked_fill(mask, value=torch.tensor(-1e9))
"""
tensor b is:
tensor([[[-1000000000, -1000000000, -1000000000, -1000000000],
         [-1000000000, -1000000000, -1000000000, -1000000000],
         [          7,           7,           7,           7]],

        [[          1,           1,           1,           1],
         [-1000000000, -1000000000, -1000000000, -1000000000],
         [-1000000000, -1000000000, -1000000000, -1000000000]]])
"""
c = a.masked_fill(mask == 0, value=torch.tensor(-1e9))  # mask等于0的对应的行列元素赋值为value
"""
tensor c is:
tensor([[[          5,           5,           5,           5],
         [          6,           6,           6,           6],
         [-1000000000, -1000000000, -1000000000, -1000000000]],

        [[-1000000000, -1000000000, -1000000000, -1000000000],
         [          2,           2,           2,           2],
         [          3,           3,           3,           3]]])
"""



tensor([[[5, 5, 5, 5],
         [6, 6, 6, 6],
         [7, 7, 7, 7]],

        [[1, 1, 1, 1],
         [2, 2, 2, 2],
         [3, 3, 3, 3]]])
torch.Size([2, 3, 4])
torch.Size([2, 3, 1])


/Users/bruce/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1640811914285/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1273.)
  


'\ntensor c is:\ntensor([[[          5,           5,           5,           5],\n         [          6,           6,           6,           6],\n         [-1000000000, -1000000000, -1000000000, -1000000000]],\n\n        [[-1000000000, -1000000000, -1000000000, -1000000000],\n         [          2,           2,           2,           2],\n         [          3,           3,           3,           3]]])\n'

In [28]:
'''转换向量过程'''
"""
repeat 和 expand 两个函数的区别, 不会分配新内存,只是创建一个新的视图, 且只能扩展维度是1的张量
"""
a = torch.tensor([1, 2, 3, 4])
a1 = a.expand(8, 4)  # 不会分配新内存,只是创建一个新的视图,且只能扩展维度是1的张量
print(a1.shape)  # [8, 4]

b1 = a.repeat(3, 2)  # 沿着特定维度重复张量
print(b1.shape)  # shape =[3, 8]

torch.Size([8, 4])
torch.Size([3, 8])


In [29]:
"""
meshgrid函数使用,构成一个坐标系可用,选择两个行列坐标的值,然后最后生成的坐标
x和y轴的参数mn分别是较小的值作为行,大的作为列
"""
x = np.linspace(0, 1, 5)
y = np.linspace(0, 1, 3)
xc, yc = np.meshgrid(x, y)
print(xc)
print(yc)
xc = xc.reshape(-1, 1)
yc = yc.reshape(-1, 1)
c = np.concatenate((xc, yc), axis=1)
print(c)

[[0.   0.25 0.5  0.75 1.  ]
 [0.   0.25 0.5  0.75 1.  ]
 [0.   0.25 0.5  0.75 1.  ]]
[[0.  0.  0.  0.  0. ]
 [0.5 0.5 0.5 0.5 0.5]
 [1.  1.  1.  1.  1. ]]
[[0.   0.  ]
 [0.25 0.  ]
 [0.5  0.  ]
 [0.75 0.  ]
 [1.   0.  ]
 [0.   0.5 ]
 [0.25 0.5 ]
 [0.5  0.5 ]
 [0.75 0.5 ]
 [1.   0.5 ]
 [0.   1.  ]
 [0.25 1.  ]
 [0.5  1.  ]
 [0.75 1.  ]
 [1.   1.  ]]


In [30]:

"""
torch.floor不超过这个数的最大整数
tensor([[ 0.0461,  0.4024, -1.0115],
        [ 0.2167, -0.6123,  0.5036]])
tensor([[ 0.,  0., -2.],
        [ 0., -1.,  0.]])
"""
a = torch.randn((2, 3))
b = torch.floor(a)
print(a)
print(b)

tensor([[ 0.0930,  0.2338, -1.1586],
        [-0.6521, -1.0775,  0.1823]])
tensor([[ 0.,  0., -2.],
        [-1., -2.,  0.]])


In [31]:
"""
上采样函数
nn.ConvTranspose2d,有参数可以训练
hout = (hin-1)*stride - 2 * padding + kernel + output_padding
参考链接https://blog.csdn.net/qq_27261889/article/details/86304061

nn.Unsample 上采样没有参数,速度更快,采样策略给定

参考链接 
https://www.shuzhiduo.com/A/gGdX9OPWz4/
https://blog.csdn.net/wangweiwells/article/details/101820932
https://zhuanlan.zhihu.com/p/87572724(详解 align_corners=False, True用法)

双线性插值算法
https://juejin.im/post/6844903924999127047
https://zhuanlan.zhihu.com/p/110754637
"""
input = torch.ones((2, 2, 3, 4))
output = nn.ConvTranspose2d(2, 4, kernel_size=4, stride=1, padding=0, bias=False)
# [2, 4, 6, 7]
print(output(input).shape)

input = torch.arange(1, 5, dtype=torch.float32).view(1, 1, 2, 2)
m = nn.Upsample(scale_factor=2, mode="nearest")
output = m(input)
print(output)
m = nn.Upsample(scale_factor=2, mode='bilinear',align_corners=False)
output = m(input)

torch.Size([2, 4, 6, 7])
tensor([[[[1., 1., 2., 2.],
          [1., 1., 2., 2.],
          [3., 3., 4., 4.],
          [3., 3., 4., 4.]]]])
